# 制图模块

> 时间：					阅读时间：

`Arcpy.mp` 、主要是用于操作现有工程 (.aprx) 和图层文件 (.lyrx) 的内容。 设计原则是**使用ArcGIS Pro应用程序创作工程及其内容，然后使用 `arcpy.mp` 自动执行重复性任务**，例如修改地图属性、添加图层、应用符号系统和导出布局。可以自动化工程的内容，甚至无需打开应用程序。 

> ArcGIS Desktop 10.x中ArcPy中的模块被称为 arcpy.mapping ，但ArcGIS Pro的map功能发生了重大更改！！！

通过描述 arcpy.mp 执行的一些方案来了解其功能将会更容易些。 下面是 arcpy.mp Python 脚本可帮助您完成的其中一些工作流：

- 自动导出布局、地图视图、地图系列和报表
- 构建多种 PDF 地图册。*例如，生成描述工程内容（包括地图、图层、表和布局）的报告。又比如参考地图册或具有标题页、多个地图页以及支持内容（如表格式报表和联系人列表）的众多附加页的专题地图册。*
- 操作地图上的一些元素，*例如更新图片、公司徽标，在工程的多个布局中搜索和替换文本字符串等。*
- 扩展地理处理脚本工具以直接使用地图、布局和其他工程元素。
- 使用库存脚本记录有关工程、数据、断开的数据源、文件夹连接等的信息。
- **更新、修复或替换图层数据源。**
- 管理布局、地图、报表、文件夹连接、视图等工程工程。
- **修改图层的符号系统。**
- 自动在线执行共享要托管在组织中的工程项。

## 理解和引用工程（Projects）

> ArcGIS Pro 将您的 GIS 项目组织到[工程](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/projects/what-is-a-project.htm)中。 工程可以包含地图、场景、布局、报表、图表以及空间和非空间数据的其他制图表达。 工程还包含与诸如系统文件夹、数据库、工具箱、服务器、样式和[活动门户](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/projects/available-online-resources.htm)等数据资源的连接。

ArcGIS Pro中的**工程**以`.aprx`文件的形式存储在磁盘上，例如`C：\Mapping\Study.aprx`。ArcPy制图模块允许您引用和操作`.aprx`文件以及包含各个图层特性的图层文件`（.lyrx）`。

要引用工程有两种方法，一是指定磁盘上现有的`.aprx`文件，二是在程序中使用时，直接引用当前ArcGIS PRO窗口中的工程，所以不适用于独立的IDE中的脚本。`arcpy.mp.ArcGISProject()` 函数实现了这两个功能。例如：

In [ ]:
# 方式一
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Study.aprx") # 用绝对路径。只有脚本和工程文件在同一个文件夹中时，可以通过设置工作空间进行相对引用。
# 方式二
aprx = arcpy.mp.ArcGISProject("CURRENT")

可以通过`save()`和`saveACopy()`保存。

> 当在python中引用ArcGISProject对象是，工程文件会被锁定，可以用`del aprx`来释放此对象。脚本运行完之后，python回自动删除对对象的引用，所以del语句不是必须得。

## 工程的属性和方法

**创建 ArcGISProject 对象后，可以读取和修改工程的属性。**，通常包括描述性属性：例如 activeMap （与聚焦视图关联的映射）、 dateSaved （上次保存工程的日期）、 documentVersion （上次保存文档时的版本）和 filePath （完整的工程路径和文件名）、defaultGeodatabase （工程的默认地理数据库位置）、 defaultToolbox （工程的默认工具箱）和 homeFolder （工程的主文件夹位置）

使用 `importDocument()` 方法可以将地图文档（.mxd文件）、地球仪文档（.3dd文件）和场景文档（.sxd）导入到工程中。此方法的语法如下：

In [ ]:
ArcGISProject.importDocument(document_path, {include_layout},                              {reuse_existing_maps})

> ArcGIS Desktop 10.x使用单独的应用程序来处理这些文档-即，ArcMap、ArcGlobe和ArcScene。这些应用程序的功能已集成到ArcGIS Pro中。还可以导入地图文件（.mapx）、布局文件（.pagx）和报告文件（.rptx）的内容。

.mxd文件中的每个数据帧都将成为.aprx文件中的地图。此地图包括所有图层及其符号系统。.mxd文件中的唯一布局将成为.aprx文件中的布局。此布局包含所有布局元素，包括比例尺、图例等。

## 理解和使用地图(map)

一个工程（project）包含一个或多个地图，每个地图通常包含一个或多个图层和表。

使用`ArcGISProject` 类的 `listMaps()` 方法访问映射：

In [ ]:
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
maps = aprx.listMaps() 
# 可以通过wild_card属性过滤，不区分大小写
# 使用maps = aprx.listMaps(“P*”)来选择P开头的地图

for m in maps:    
	print(m.name)
del aprx

地图、图层和表格是在列表中使用的理想对象，有助于自动执行任务。

#### 使用地图对象的属性

#### 使用地图对象的方法

用来添加、移除或重新排列图层和表

## 理解和使用图层

ArcGIS Pro中的地图通常包含一个或多个图层。图层是对数据源（如shapefile、地理数据库要素类或栅格）的引用，用于定义数据在地图上的符号化方式。 arcpy.mp 中的两个类与层一起工作： `Layer` 和 `LayerFile` 这两种类型的对象都用于管理图层。

可以使用 `Map.listLayers()` 方法访问地图中的图层。语法`Map.listLayers({wildcard})`。方法返回一个 Layer 对象的列表，这些对象具有属性和方法。例如，下面的代码遍历项目中的地图，然后打印每个地图中所有图层的名称：

In [ ]:
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
maps = aprx.listMaps()
for m in maps:    
	print("Map: " + m.name)    
	lyrs = m.listLayers()    
	for lyr in lyrs:        
		print(lyr.name)
del aprx 

Layer 对象有许多属性： `name` 、 `dataSource`、`brightness`、`contrast`、`showLabels`、`symbology`、`transparency`和`visible`。

> 拓展：
>
> ArcGIS Pro include 3D layer, basemap layer, feature layer, group layer, network analyst layer, network dataset layer, raster layer, and web layer.
>
> using an is* property—i.e., is3DLayer, isBasemapLayer, isFeatureLayer, and so on.
>
> 确定图层类型的另一种方法是在尝试将属性应用于某个图层之前，在 Layer 对象上使用 `supports()` 方法来测试该图层是否支持特定属性。supports() 方法的语法是`Layer.supports(layer_property)`，返回布尔值。使用此方法避免脚本出错。

## 7.添加、删除、插入和移动图层

### 1）给地图添加底图：

In [ ]:
Map.addBasemap(basemap_name)

> basemap_name 参数基于底图库中的显示名称（即，图像），而不是图层被添加到地图之后的名称（即，World Imagery）。![image-20230811114157461](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/image-20230811114157461.png)

可以使用 `Map.removeLayer()` 方法删除底图。

### 2）给地图添加数据

在ArcGIS Pro中，可以通过从目录窗格中拖动数据集或单击地图选项卡上的添加数据来将数据添加到地图。此功能在ArcPy中使用 `Map.addDataFromPath()` 方法复制。此方法的语法为

```
Map.addDataFromPath(data_path)
```

`data_path` 参数是表示本地路径或URL的字符串。以下代码将要素类添加到现有地图：

```
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
m = aprx.listMaps("Parks")[0]
m.addDataFromPath("C:/Mapping/Data.gdb/sidewalks")
aprx.save()
del aprx
```

通过创建图层对象来删除添加的数据：

```
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
m = aprx.listMaps("Parks")[0]
lyr = m.listLayers("historical_landmarks")[0]
m.removeLayer(lyr)
aprx.save()
del aprx
```

除了通过引用磁盘上的数据向地图添加新图层外，还可以通过引用 Layer 或 LayerFile 对象添加图层。您可以使用 `Map.addLayer()` 和 `Map.insertLayer()` 方法添加图层。这些方法以类似的方式工作，但后者提供了对图层放置的更细粒度的控制。这两个方法中的第一个方法的语法为`Map.addLayer(add_layer_or_layerfile, {add_position})`

> 此语法与 addDataFromPath() 方法类似，但.lyrx文件不仅包含对源数据的引用，还包含由图层文件的作者创建的自定义符号系统。可以使用保存到图层文件地理处理工具创建.lyrx文件。创建图层文件通常用于在项目之间和用户之间共享符号系统。

`Map.addLayer()` 方法也可以使用 `Layer` 对象而不是 `LayerFile` 对象。此方法的典型应用是引用一个地图中的图层，然后将其添加到同一项目中的另一个地图中。以下代码引用公园地图中名为city_parks的图层，然后将此图层添加到设施地图中：

In [ ]:
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
m = aprx.listMaps("Parks")[0]
lyr = m.listLayers("city_parks")[0]
m = aprx.listMaps("Facilities")[0]
map.addLayer(lyr, "BOTTOM")
aprx.save()
del aprx

## 使用图层符号系统

> 如果你需要可视化调试更加美观的符号系统，还是选择程序中操作，以下仅适用于符号系统的样式最终确定之后，利用符号系统的脚本实现自动化流程。

使用 `Layer` 对象时，将保留其符号系统。例如，当引用一个地图中的图层并使用 `Map.insertLayer()` 方法将图层添加到另一个地图时，符号系统是相同的。

> `Map.insertLayer()` 很像 `Map.addLayer()` 方法。

此外，还可以通过 Layer 对象的属性（包括亮度、对比度和透明度）修改符号系统的某些方面。这些属性是可读写的。然而，符号系统的许多其他方面需要更细粒度的控制。

### 符号系统

使用符号系统是通过 `Symbology` 类完成的，可以通过 `Layer` 对象的 `symbology` 属性访问该类。在典型的工作流中，可以引用图层的 `symbology` 属性，对 `Symbology` 对象进行更改，然后将这些更改应用于图层。

`Symbology` 类具有两个属性，用于定义图层的符号化方式： `colorizer` （用于栅格层）和 renderer （用于要素图层）。这些属性返回用于符号化图层的*着色器*或*渲染器*。还可以使用 `updateColorizer()` （用于栅格图层）和 `updateRenderer()` （用于要素图层）方法更改着色器或渲染器的类型。

---

下面的示例代码迭代地图中的图层。

In [ ]:
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Symbology.aprx")
m = aprx.listMaps("Plan")[0]
lyrs = m.listLayers()
for lyr in lyrs:
  # 使用 Layer 对象的 symbology 属性创建一个 `Symbology` 对象
	sym = lyr.symbology
	
	# 检查图层的类型
	if lyr.isFeatureLayer:
  	if hasattr(sym, "renderer"): # 图层是否支持着色器
    	print(lyr.name + ": " + sym.renderer.type)
  if lyr.isRasterLayer:
  	if hasattr(sym, "colorizer"): # 图层是否支持渲染器
    	print(lyr.name + ": " + sym.colorizer.type)

---

以下代码使用RGB（红色、绿色、蓝色）值更新多段线的颜色：

In [ ]:
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Symbology.aprx")
m = aprx.listMaps("Plan")[0]
lyr = m.listLayers("trails")[0]
sym = lyr.symbology
red = {"RGB": [255, 0, 0, 100]}
	if lyr.isFeatureLayer and hasattr(sym, "renderer"):
    
    # 将RGB值分配给 Symbol 对象的 color 属性
		sym.renderer.symbol.color = red
    
    # 将新符号系统应用于lyr图层的符号系统属性
		lyr.symbology = sym
    
aprx.save()
del aprx

程序中的结果：

---

**可以使用符号库中预定义的样式名称：**

![image-20230811121409439](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/image-20230811121409439.png)

使用 Symbol 类的 `applySymbolFromGallery()` 方法更新符号系统：`sym.renderer.symbol.applySymbolFromGallery("Dashed 4:4")`。

> 如果符号同名使用该方法的可选参数：
>
> ![image-20230811121607536](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/image-20230811121607536.png)
>
> ```
> sym.renderer.symbol.applySymbolFromGallery("Hospital", 1) # 选择第二个医院符号（从0开始）
> ```
>
> 

---

### 应用分级颜色符号系统

使用 Symbol.updateRenderer() 方法来更改图层的渲染器，

下面的示例使用空置房屋的原始计数（在名为VACANT的字段中），并通过所有房屋单元的原始计数（在名为HSE_UNITS的字段中）对该计数进行归一化，得到一个分数。使用五个类别应用等间隔分类：

sym.renderer.classificationField = "VACANT"    sym.renderer.normalizationField = "HSE_UNITS"    sym.renderer.classificationMethod = "EqualInterval"    sym.renderer.breakCount = 5


![image-20230811122419694](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/image-20230811122419694.png)

```
sym.renderer.colorRamp = aprx.listColorRamps("Yellow-Orange-Red                                                  (5 Classes)")[0]
```

最后，使用 lyr.symbology = sym 将新符号系统应用于图层，并保存项目以使更改生效：

```
lyr.symbology = symaprx.saveACopy("Housing_Choropleth.aprx")del aprx
```

参考官方文档写一个案例。




## 将符号系统应用于栅格图层

可以对土地利用图进行映射




## 使用布局（layout）

[页面布局](https://pro.arcgis.com/zh-cn/pro-app/latest/help/layouts/layouts-in-arcgis-pro.htm)（通常简称为布局）是在虚拟页面上组织的[地图元素（element）](https://pro.arcgis.com/zh-cn/pro-app/latest/help/layouts/work-with-layout-elements.htm)的集合，旨在用于地图打印。常见的地图元素包括一个或多个[地图框](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/add-and-modify-map-frames.htm)（每个地图框都含有一组有序的地图图层）、[比例尺](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/scale-bars.htm)、[指北针](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/north-arrows.htm)、地图标题、描述性文本和[图例](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/work-with-a-legend.htm)。为提供地理参考，可以[添加格网](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/add-and-modify-grids.htm)或[经纬网](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/layouts/add-and-modify-graticules.htm)。

![布局元素概述](https://image-1315363329.cos.ap-shanghai.myqcloud.com/lessons/GUID-5B8FE672-EE85-49A7-A5B3-FAC97FE07492-web.png)

> 和使用地图类型。

`ArcGISProject.listLayouts({wildcard})`

该方法返回一个 Layout 对象的列表，每个对象引用一个单页布局。

`Layout` 对象提供对布局的名称、页面大小和页面单位等属性的访问。特定页面布局由 name 属性引用。因此，项目中的每个布局都有唯一的名称非常重要。 Layout 对象还提供了对布局的所有元素以及几种不同的导出方法的访问。

上文说的地图框、文本等页面布局元素可以用过此函数引用：`Layout.listElements ({element_type}, {wildcard})`。之后可以对返回的元素对象进行例如名称、高度、宽度、位置、旋转、可见性进行修改。

## 使用页面上地图、地图框、相机

首先得理解其概念：

地图框存在于布局中，用来限制地图图层的展示边界，就像打开了另一个地图视图一样。地图框可以指向工程中的任何[地图](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/mapping/map-authoring/maps.htm)或[场景](https://pro.arcgis.com/zh-cn/pro-app/3.1/help/mapping/map-authoring/scenes.htm)，也可以完全不指向任何地图。 创建地图框后，可以随时更新其指向的地图。

camera 属性。此属性允许访问 Camera 对象，该对象控制地图框中显示的数据的位置和查看位置。Camera 对象控制视图的XYZ值。对于二维贴图，XY值基于贴图框的中心，而Z值不受支持。对于三维贴图，XYZ值表示相机相对于视图的位置。 Camera 对象还包括一个比例属性，该属性仅适用于2D地图，因为其平面视图。

常见的任务是使多个地图帧的范围相同，这些方法可用于此任务。下面的代码示例采用一个地图框的范围，并将相同的范围应用于同一布局中的第二个地图框：

```
import arcpy
aprx = arcpy.mp.ArcGISProject("C:/Mapping/Demo.aprx")
lyt = aprx.listLayouts("Parks")[0]
mf1 = lyt.listElements("MAPFRAME_ELEMENT", "Park A")[0]
ext = mf1.camera.getExtent() # 由左下角和右上角的坐标
# 可以使用 MapFrame.getLayerExtent() 方法获取图层的范围。空间数据集（包括要素类和栅格）的范围可以使用 Describe() 和 da.Describe() 函数获得。
mf2 = lyt.listElements("MAPFRAME_ELEMENT", "Park B")[0]
mf2.camera.setExtent(ext)
aprx.save()
del aprx
```


## 导出和打印布局

以使用与 Layout 对象关联的导出方法来导出布局。每种文件格式都有不同的方法，如下所示，常用的`exportToJPEG()`、`exportToPNG()`、`exportToPDF()`和`exportToTIFF()`。

```
# 以到处jpg为例，具体翻阅文档
exportToJPEG(out_jpg, {resolution}, {jpeg_color_mode},
{jpeg_quality}, {embed_color_profile},              
{clip_to_elements})
```

> JPG和PNG的差别：JPEG文件是网络上流行的格式，因为文件大小比许多其他格式都小，但该算法使用有损压缩，这意味着原始图像中的一些数据会丢失，绘图和文本等元素可能会变得模糊。更广泛推荐的地图图像格式是PNG。
>
> PDF格式的优点：已成为一种广泛使用的导出布局格式，部分原因是它可以跨不同平台查看和打印。PDF文件也可以在其他应用程序中编辑，并保留布局中的大部分信息，包括地理配准信息、注释和标签。

---

拓展：由于PDF文档被广泛用于导出布局，因此 arcpy.mp 包含了一个 PDFDocument 类来管理PDF文件，包括合并多个文件、删除页面和管理文档安全设置。 PDFDocument 类只有一个属性： pageCount ，它是页数的整数。 PDFDocument 类有六个方法： appendPages() 、 deletePages() 、 insertPages() 、 saveAndClose() 、 updateDocProperties() 和 updateDocSecurity() 。

arcpy.mp 中的两个函数有助于处理PDF文档。 PDFDocumentOpen () 函数通过指向磁盘上的PDF文件返回 PDFDocument 对象，而 PDFDocumentCreate() 函数在内存中创建新的 PDFDocument 对象。使用这些函数的典型场景是创建PDF格式的地图册。您可以使用 PDFDocumentCreate() 函数在内存中创建一个新的（空的） PDFDocument ，然后使用 PDFDocument 对象的 appendPages() 或 insertPages() 方法添加页面。

下面的代码使用 PDFDocumentCreate () 函数创建一个空的 PDFDocument ，并将三个现有的PDF文件附加到内存中的此文档中。 saveAndClose() 方法保存生成的PDF，如下所示：

```
import arcpy
pdfpath = "C:/Mapping/MapBook.pdf"
pdfdoc = arcpy.mp.PDFDocumentCreate(pdfpath) # PDFDocumentCreate() 函数不会创建任何空白PDF页面。
pdfdoc.appendPages("C:/Mapping/Cover.pdf")
pdfdoc.appendPages("C:/Mapping /Map1.pdf")
pdfdoc.appendPages("C:/Mapping /Map2.pdf")
pdfdoc.saveAndClose()
del pdfdoc
```


#### *实操：制作各省份的市级的人口密度图。*

#### 实操：批量导出地图